In [1]:
import json
import re
import pandas as pd
import urllib
from tqdm import tqdm
from urllib.request import Request
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
def parseData(offer):
    dto = {}
    try:
        dto['name'] = offer['name']
    except KeyError:
        dto['name'] = None
    try:
        dto['voivodeship'] = offer['itemOffered']['address']['addressRegion']
    except KeyError:
        dto['voivodeship'] = None
    try:
        dto['city'] = offer['itemOffered']['address']['addressLocality']
    except KeyError:
        dto['city'] = None
    try:
        dto['price'] = offer['price']
    except KeyError:
        dto['price'] = None
    try:
        dto['currency'] = offer['currency']
    except KeyError:
        dto['currency'] = None
    try:
        dto['rooms'] = offer['itemOffered']['numberOfRooms']
    except KeyError:
        dto['rooms'] = None
    try:
        dto['size'] = offer['itemOffered']['floorSize']['value']
    except KeyError:
        dto['size'] = None
    return dto

In [23]:
url = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/cala-polska?page=1&limit=36&ownerTypeSingleSelect=ALL&by=DEFAULT&direction=DESC&viewType=listing&page='

apartaments = []
# for page_number in tqdm(range(1,3000)):
with urllib.request.urlopen(f'{url}{1}') as resp:
    page = BeautifulSoup(resp.read().decode('utf-8'), "html.parser")
    html = page.find(attrs={'type' : 'application/ld+json'})
    data = json.loads(html.text)
    for offer in data['@graph'][2]['offers']['offers']:
        apartaments.append(parseData(offer))

KeyError: 'currency'

In [18]:
data['@graph'][2]['offers']['offers'][0]

{'@type': 'Offer',
 'availability': 'InStock',
 'image': 'https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Imd2NnltYXQ0NDBjaTMtQVBMIiwidyI6W3siZm4iOiJlbnZmcXFlMWF5NGsxLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.qZiNpt4pYeiQIAPdO8L0Nw6Kn1-bKUAXFL23uzOQ5W8/image;s=655x491;q=80',
 'name': 'Mieszkanie 2 pok. 48,62m2 ul. Juliusza Lea',
 'price': 719000,
 'priceCurrency': 'PLN',
 'url': 'https://www.otodom.pl/pl/oferta/mieszkanie-2-pok-48-62m2-ul-juliusza-lea-ID4lnOW',
 'itemOffered': {'@type': 'Accommodation',
  'description': 'Świetne 2 pokojowe mieszkanie na sprzedaż w miejscowości Kraków, Krowodrza, za cenę 719 000 zł. To mieszkanie na sprzedaż  na 3 piętrze ma 48,62 m² powierzchni użytkowej i 48,62 m² powierzchni całkowitej. Biuro nieruchomości Blackhouse Realty jako najważniejsze zalety mieszkania wymienia: balkon, piwnica, garaż/miejsce parkingowe. Otodom 64201801',
  'address': {'@type': 'PostalAddress',
   'addressCountry': 'Polska',
   'addressLocality': 'Kraków',
   'add